In [1]:
import pendulum
import yfinance as yf
import backtrader as bt

In [2]:
class GridStrategy(bt.Strategy):
    params = dict(
        long_bench=-0.02,
        short_bench=0.05,
    )

    lines = ("num_long",)

    def __init__(self):
        self.base_price = None

    def log(self, txt, dt=None):
        print(txt)

    def next(self):
        # set current price as base price if not traded yet
        if not self.base_price:
            self.base_price = self.data.close[0]
            self.log(f"Setup base price as {self.base_price}")

        unit_value = self.broker.get_cash() / 10
        data_value = self.broker.get_value([self.data])
        returns = (self.data.close[0] - self.base_price) / self.base_price
        
        # TODO: stop loss
        if self.get_returns(days=10) < -0.1 and self.position:
            self.stop_loss()

        if self.position:
            if returns > self.p.short_bench and data_value > 6 * unit_value:
                self.order_target_value(target=6 * unit_value)
                self.base_price = self.data.close[0]
                self.log(f"sell at {self.base_price}, current value: {self.broker.get_value([self.data])}")
                
            elif returns > 2 * self.p.short_bench and data_value > 3 * unit_value:
                self.order_target_value(target=3 * unit_value)
                self.base_price = self.data.close[0]
                self.log(f"sell at {self.base_price}, current value: {self.broker.get_value([self.data])}")
                
            elif returns > 3 * self.p.short_bench and data_value > 1 * unit_value:
                self.order_target_value(target=1 * unit_value)
                self.base_price = self.data.close[0]
                self.log(f"sell at {self.base_price}, current value: {self.broker.get_value([self.data])}")
                
            elif returns > 4 * self.p.short_bench and data_value > 0:
                self.order_target_value(target=0)
                self.base_price = self.data.close[0]
                self.log(f"sell at {self.base_price}, current value: {self.broker.get_value([self.data])}")
                

        if self.broker.cash > 0:
            if returns < self.p.long_bench and data_value < 4 * unit_value:
                self.order_target_value(target=4 * unit_value)
                self.base_price = self.data.close[0]
                self.log(f"buy at grid 1, price: {self.base_price}, current value: {self.broker.get_value([self.data])}")
            elif returns < 2 * self.p.long_bench and data_value < 7 * unit_value:
                self.order_target_value(target=7 * unit_value)
                self.base_price = self.data.close[0]
                self.log(f"buy at grid 2, price: {self.base_price}, current value: {self.broker.get_value([self.data])}")
                
            elif returns < 3 * self.p.long_bench and data_value < 9 * unit_value:
                self.order_target_value(target=9 * unit_value)
                self.base_price = self.data.close[0]
                self.log(f"buy at grid 3, price: {self.base_price}, current value: {self.broker.get_value([self.data])}")
                
            elif returns < 4 * self.p.long_bench and data_value < 10 * unit_value:
                self.order_target_value(target=10 * unit_value)
                self.base_price = self.data.close[0]
                self.log(f"buy at grid 4, price: {self.base_price}, current value: {self.broker.get_value([self.data])}")
                
    def get_returns(self, days=5):
        return (self.data[0]- self.data[-days+1]) / self.data[-days+1]

    def stop_loss(self, threshold=0.1):
        self.log("liquidate position")
        self.log(f"Current value: {self.broker.get_value([self.data])}")
        self.order_target_value(target=0)
        self.base_price = self.data[0]
        self.log(f"set base price as {self.base_price}")

In [3]:
start = pendulum.parse("2023-01-01")
end = pendulum.parse("2023-07-01")
df = yf.download(
    tickers="MSFT",
    interval="1h",
    start=start,
    end=end,
)

cerebro = bt.Cerebro()
cerebro.addwriter(bt.WriterFile)
cerebro.addstrategy(GridStrategy)

data = bt.feeds.PandasData(
    dataname=df,
    timeframe=bt.TimeFrame.Minutes,
    compression=60,
)

cerebro.adddata(data)
result = cerebro.run()

[*********************100%***********************]  1 of 1 completed
Setup base price as 239.78810119628906
buy at grid 1, price: 227.14999389648438, current value: 0.0
liquidate position
Current value: 3878.248992919922
set base price as 228.13229370117188
buy at grid 1, price: 223.11900329589844, current value: 0.0
sell at 234.27999877929688, current value: 3982.759979248047
sell at 247.0, current value: 3952.0
sell at 261.1400146484375, current value: 4178.240234375
buy at grid 2, price: 249.3800048828125, current value: 3740.7000732421875
sell at 262.3800048828125, current value: 4722.840087890625
sell at 288.6600036621094, current value: 4041.2400512695312
buy at grid 1, price: 282.6300048828125, current value: 2261.0400390625
sell at 310.9800109863281, current value: 3731.7601318359375
sell at 344.4599914550781, current value: 2755.679931640625
buy at grid 1, price: 336.3800048828125, current value: 2691.0400390625
Cerebro:
  ------------------------------------------------------

In [4]:
cerebro.plot(
    iplot=False,
    #  Format string for the display of ticks on the x axis
    fmt_x_ticks="%Y-%b-%d %H:%M",
    # Format string for the display of data points values
    fmt_x_data="%Y-%b-%d %H:%M",
)

[[<Figure size 1698x1016 with 4 Axes>]]